In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import math
from pprint import pprint
import csv
import random
import requests, re, json
from bs4 import BeautifulSoup

In [2]:
'''
# Define function to generate the dataframe for EDA
def create_transfer_eda_df(to_club_name, transfer_season):
    transfers_df = pd.read_csv('transfers.csv')
    games_df = pd.read_csv('games.csv')
    # Step 1: Filter transfers for the specified club and season
    filtered_transfers = transfers_df[(transfers_df['to_club_name'] == to_club_name) & (transfers_df['transfer_season'] == transfer_season)]
    
    # Step 2: Filter games for the specified season, competition 'GR1', and round '38. Matchday'
    filtered_games = games_df[(games_df['season'] == int(transfer_season[:2] + transfer_season[3:])) &
                              (games_df['competition_id'] == 'GR1') &
                              (games_df['round'] == '38. Matchday')]
    
    # Step 3: Select relevant columns from filtered_games
    filtered_games = filtered_games[['season', 'home_club_id', 'away_club_id', 
                                     'home_club_position', 'away_club_position']]
    
    # Step 4: Merge on club_id, considering both home and away positions
    merged_df = pd.merge(filtered_transfers, filtered_games, left_on='to_club_name', right_on='home_club_id', how='left')
    merged_df = merged_df.rename(columns={'home_club_position': 'table_position'}).drop(columns=['home_club_id', 'away_club_id'])
    
    # Final selection of columns to display
    final_df = merged_df[['player_id', 'transfer_fee', 'market_value_in_eur', 'to_club_name', 'table_position']]
    return final_df
'''

"\n# Define function to generate the dataframe for EDA\ndef create_transfer_eda_df(to_club_name, transfer_season):\n    transfers_df = pd.read_csv('transfers.csv')\n    games_df = pd.read_csv('games.csv')\n    # Step 1: Filter transfers for the specified club and season\n    filtered_transfers = transfers_df[(transfers_df['to_club_name'] == to_club_name) & (transfers_df['transfer_season'] == transfer_season)]\n    \n    # Step 2: Filter games for the specified season, competition 'GR1', and round '38. Matchday'\n    filtered_games = games_df[(games_df['season'] == int(transfer_season[:2] + transfer_season[3:])) &\n                              (games_df['competition_id'] == 'GR1') &\n                              (games_df['round'] == '38. Matchday')]\n    \n    # Step 3: Select relevant columns from filtered_games\n    filtered_games = filtered_games[['season', 'home_club_id', 'away_club_id', \n                                     'home_club_position', 'away_club_position']]\n    \n  

In [3]:
'''
to_club_id = 27  # Example club ID
transfer_season = "25/26"  # Example season format

# Generate the EDA dataframe
transfer_eda_df = create_transfer_eda_df(to_club_id, transfer_season)
transfer_eda_df
'''

'\nto_club_id = 27  # Example club ID\ntransfer_season = "25/26"  # Example season format\n\n# Generate the EDA dataframe\ntransfer_eda_df = create_transfer_eda_df(to_club_id, transfer_season)\ntransfer_eda_df\n'

In [26]:
def create_transfer_dataframe(club_name, season):
    # Read the CSV files
    transfers_df = pd.read_csv('transfers.csv')
    games_df = pd.read_csv('games.csv')
    
    # Filter transfers for the given club and season
    club_transfers = transfers_df[(transfers_df['to_club_name'] == club_name) & 
                                  (transfers_df['transfer_season'] == season)]
    
    if club_transfers.empty:
        print(f"No transfers found for {club_name} in season {season}")
        return pd.DataFrame()
    
    # Get the club_id
    club_id = club_transfers['to_club_id'].iloc[0]
    
    # Filter games for the club, season, and specific competition and round
    season_year = int(season.split('/')[0]) + 2000  # Convert season to year
    club_games = games_df[(games_df['home_club_id'] == club_id) | 
                          (games_df['away_club_id'] == club_id)]
    club_games = club_games[(club_games['season'] == season_year) & 
                            (club_games['competition_id'] == 'GB1') & 
                            (club_games['round'] == '38. Matchday')]
    
    # Get the club's position
    if not club_games.empty:
        if club_games['home_club_id'].iloc[0] == club_id:
            position = club_games['home_club_position'].iloc[0]
        else:
            position = club_games['away_club_position'].iloc[0]
    else:
        print(f"No Premier League fixtures found for {club_name} in season {season}")
        position = np.nan
    
    # Create the final dataframe
    result_df = pd.DataFrame({
        'player_id': club_transfers['player_id'],
        'transfer_fee': club_transfers['transfer_fee'],
        'market_value_in_eur': club_transfers['market_value_in_eur'],
        'club_id': club_id,
        'epl_position': position
    })
    
    return result_df

In [27]:
# Example usage
club_name = "Wolves"
season = "18/19"
df = create_transfer_dataframe(club_name, season)
print(df)

       player_id  transfer_fee  market_value_in_eur  club_id  epl_position
41999     233124           0.0            1000000.0      543           7.0
42270     361254           0.0            3000000.0      543           7.0
42987     175446    21000000.0           12000000.0      543           7.0
43699     180843           0.0             500000.0      543           7.0
44724     168157           0.0           17000000.0      543           7.0
44746     525247     4750000.0                  NaN      543           7.0
44757     204103    20000000.0            8000000.0      543           7.0
45038     175446           0.0           12000000.0      543           7.0
45083      29364     5600000.0           10000000.0      543           7.0
45604      45026    18000000.0           16000000.0      543           7.0
45743     142310    12000000.0            5000000.0      543           7.0
45839     206040           0.0           15000000.0      543           7.0
46136     340950    14000